## Demo : Running vcluster in rootless mode

### Create vcluster

##### Values.yaml preparation

In [1]:
! cat values.yaml

# Refer https://github.com/loft-sh/vcluster/blob/main/charts/k3s/values.yaml for full set of customisable options

# Run as a non-root user, guaranteed to work only for k3s
securityContext:
  runAsUser: 12345
  runAsNonRoot: true

##### Create vcluster with the customized values.yaml

In [2]:
! vcluster create vc1 -n host-namespace -f values.yaml --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/595511949 --namespace host-namespace --install --repository-config='' --values /tmp/102398210 --values values.yaml
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [15]:
! kubectl get all -n host-namespace

NAME                                              READY   STATUS    RESTARTS   AGE
pod/coredns-7c65b46c6-8ppwr-x-kube-system-x-vc1   1/1     Running   0          7s
pod/vc1-0                                         2/2     Running   0          23s

NAME                                         TYPE           CLUSTER-IP      EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1         ClusterIP      10.96.44.94     <none>           53/UDP,53/TCP,9153/TCP   7s
service/vc1                                  LoadBalancer   10.96.62.8      172.18.255.201   443:31144/TCP            23s
service/vc1-headless                         ClusterIP      None            <none>           443/TCP                  23s
service/vc1-node-kcd-vcluster-demo-worker3   ClusterIP      10.96.195.106   <none>           10250/TCP                7s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     23s


##### Accessing Vcluster

In [16]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


##### Verify that pod vc1-0 is running as a non-root user

In [17]:
! kubectl exec vc1-0 --namespace host-namespace -- id 

Defaulted container "vcluster" out of: vcluster, syncer
uid=12345 gid=0 groups=0


### Cleanup

In [18]:
! vcluster delete vc1 -n host-namespace 
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/4005648566 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [19]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   45m
kube-node-lease      Active   45m
kube-public          Active   45m
kube-system          Active   45m
local-path-storage   Active   45m
metallb-system       Active   44m
